# Part I. ETL Pipeline for Pre-Processing the Files

## THE CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# return the columns'name and index to facilitate future processing
data = pd.read_csv('event_datafile_new.csv')
print(
    [(idx,col) for idx,col in enumerate(data.columns)],
    data.columns)

[(0, 'artist'), (1, 'firstName'), (2, 'gender'), (3, 'itemInSession'), (4, 'lastName'), (5, 'length'), (6, 'level'), (7, 'location'), (8, 'sessionId'), (9, 'song'), (10, 'userId')] Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')


# Part II. Apache Cassandra coding portion of the project. 

## Now it is ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# make a connection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
## For table session_lib, the sessionId, itemInSession were used as a partition key because the queries will filter by these columns.

query = "CREATE TABLE IF NOT EXISTS session_lib "
query = query + "( sessionId int,itemInSession int, artist text, song_title text, song_length float,\
        PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# insert data into the table created above
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_lib (sessionId,itemInSession,artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length FROM session_lib  WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,",",row.song_title,",",row.song_length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [13]:
## For table user_lib, the userId, sessionId were used as a partition key because the queries will filter by these columns.\
## itemInSession was used as clustering columns to help make up a unique key.

query = "CREATE TABLE IF NOT EXISTS user_lib "
query = query + "( userId int,sessionId int,itemInSession int,artist text, song_title text,firstName text,lastName text,  \
        PRIMARY KEY ((userId, sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [14]:
# insert data into the table created above
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_lib (userId,sessionId,itemInSession,artist, song_title,firstName,lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9], line[1], line[4]))

In [15]:
## Add in the SELECT statement to verify the data was entered into the table:

query = "SELECT artist, song_title,firstName,lastName FROM user_lib  WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,",",row.song_title,",",row.firstname,",",row.lastname)

Down To The Bone , Keep On Keepin' On , Sylvie , Cruz
Three Drives , Greece 2000 , Sylvie , Cruz
Sebastien Tellier , Kilometer , Sylvie , Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie , Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [21]:
## For table song_lib, the song_title,artist were used as a partition key because the queries will filter by this column.

query = "CREATE TABLE IF NOT EXISTS song_lib "
query = query + "( song_title text,artist text, firstName text,lastName text,sessionId int,itemInSession int, userId int, \
        PRIMARY KEY (song_title,artist))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [22]:
# insert data into the table created above
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_lib ( song_title,artist,firstName,lastName,sessionId,itemInSession, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9],line[0],line[1], line[4], int(line[8]),int(line[3]),int(line[10])))

In [23]:
query = "SELECT firstName,lastName FROM song_lib  WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname,",",row.lastname)

Jacqueline , Lynch


### Drop the tables before closing out the sessions

In [24]:
## Drop the tables before closing out the sessions
query1 = "drop table if exists session_lib"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "drop table if exists user_lib"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query3 = "drop table if exists song_lib"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()